In [1]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

## 데이터 준비
- 모델에 넘겨줄 데이터 (dls):
- X (입력 데이터): 사용자 ID와 영화 ID.
- Y (타겟 데이터): 사용자별 평점 점수.

In [15]:
# path/'u.data' 기본 테이블 경로
# 사용자ID, 영화ID, 점수 등이 있어.
ratings = pd.read_csv(path/'u.data',
                     delimiter='\t',
                     header=None,
                     names=['user','movie','rating','timestamp'])

ratings.head(2)

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742


In [5]:
# 사용자 간 유사성은 어떻게 구할까?
# SF, 액션, 고전을 좋아하는 정도를 수치로 나타내.
user1 = np.array([0.98, 0.9, -0.9])
user2 = np.array([0.9, 0.8, -0.6])

# user1과 user2의 유사성은 접곰으로 구해.
(user1 * user2).sum()

2.1420000000000003

In [18]:
# 영화ID을 제목으로 대체할거야.
# 제목 경로는 path/'u.item'
movies = pd.read_csv(path/'u.item',
                     delimiter='|',
                     encoding='latin-1',
                     usecols=(0,1),
                     names=['movie','title'],
                     header=None)

movies.head(2)

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)


In [20]:
# 제목 테이블 (movies) 를 기본 테이블 (ratings) 에 결합해.

ratings = ratings.merge(movies)
ratings.head(2)

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)


## 데이터로더
- CallabDataLoaders 의 디폴트 특징
- 1번열로 user, 2번열로 항목(movie), 3번열로 rating을 가져다가 dls를 만들어.
- **2번열을 movie 대신 title로 바꾸려 해.**
- 열 변경은 user_name, item_name, rating_name 매개변수를 바꾸면 돼.

In [33]:
dls = CollabDataLoaders.from_df(ratings,
                                item_name='title',
                                bs=64)
dls.show_batch()
# dls[:, 0] = user
# dls[:, 1] = title

,user,title,rating
0,11,"Brady Bunch Movie, The (1995)",3
1,714,Contact (1997),4
2,555,One Flew Over the Cuckoo's Nest (1975),4
3,67,Dracula: Dead and Loving It (1995),3
4,450,Taxi Driver (1976),5
5,62,"Remains of the Day, The (1993)",2
6,921,Robin Hood: Men in Tights (1993),3
7,620,Richie Rich (1994),4
8,128,This Is Spinal Tap (1984),4
9,785,Scream (1996),3


## 모델이 수행할 작업
- **임베딩**: 사용자 ID와 영화 ID를 수치(임베딩 벡터)로 변환해.
- **점곱 계산**: 사용자의 임베딩 벡터와 영화의 임베딩 벡터를 요소별로 곱하고, 그 결과를 합산하여 사용자별 평점 **예측** 점수를 계산해.
- **손실 최소화**: 평점 **예측** 점수와 실제 점수 간의 차이(손실)를 줄이기 위해 모델을 학습해.

**임베딩**
- 범주형(ID등)을 수치형으로 바꾸는 것.
- 사용자수를 100, 잠재요소개수를 5라고 하자.

1. Embedding 계층 (user_fatcors) 초기화 : (100x5) 모양.
2. 인덱싱(Indexing) : 사용자ID 넘기기.
- Embedding 내부엔 임베딩 테이블이 있어.
- 이 테이블에는 ID 열과 벡터 열이 저장돼.
- 입력으로 제공된 사용자 ID는 인덱스로 사용돼. <br>
| ID | 벡터          |
|----|-------------|
| 0  | [x1, x2, x3] |
| 1  | [y1, y2, y3] |
| 2  | [z1, z2, z3] |

**점곱**
- user_factors (100x5) 와 movie_factors (100x5) 를 요소별로 곱하고 합산.
- 이 값이 사용자별 평점 예측 점수야. (100) 모양.
- *NumPy의 np.dot() 함수와 달리, 점곱은 동일한 위치에 있는 요소 끼리 곱해.

In [35]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_fatcors = 5

torch.randn(n_users, n_fatcors) # 임의로 만든 n_users*5 행렬.

tensor([[ 0.7052, -0.7256,  0.2316,  1.1350,  0.5629],
        [-0.4386,  0.0263, -0.3809, -0.8161, -0.8348],
        [ 1.8694, -0.2447,  2.1778, -0.8791, -0.6799],
        ...,
        [-0.2005, -2.0247,  1.6692, -0.2434,  0.1669],
        [-0.3566,  0.1073, -0.1075, -0.0984, -0.3356],
        [-0.2680, -0.6210,  2.8216,  1.4700, -0.1498]])

In [49]:
class DotProduct(Module):
    # Embedding 계층 초기화
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
    
    # 사용자 및 영화 ID 넘기기
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return (users*movies).sum(dim=1) # 점곱 계산

In [50]:
model = DotProduct(n_users, n_movies, 50)

learn = Learner(dls, model, loss_func=MSELossFlat())

In [51]:
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.369885,1.288840,00:09
1,1.051717,1.080287,00:09
2,0.895215,0.977860,00:09
3,0.824125,0.887335,00:08
4,0.772121,0.871656,00:09


## 모델 개선: 예측 범위를 0~5가 되도록 강제
- sigmoid_range 함수로 구현

In [52]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.y_range = y_range
        
    # 예측 범위를 0~5가 되도록 강제
    # sigmoid_range함수로 구현
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return sigmoid_range((users*movies).sum(dim=1), *self.y_range)

model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.995935,0.973719,00:10
1,0.883593,0.896091,00:09
2,0.680941,0.871068,00:09
3,0.477415,0.881028,00:08
4,0.351564,0.884532,00:09


## 모델 개선: 편향 추가

In [53]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias = Embedding(n_users, 1) # 사용자 편향
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1) # 영화 편향
        self.y_range = y_range
        
    
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        res = (users*movies).sum(dim=1, keepdim=True) # 점곱 계산
        res += self.user_bias(x[:, 0]) + self.movie_bias(x[:, 1]) # 편향 더하기
        return sigmoid_range(res, *self.y_range)

model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.933222,0.935297,00:10
1,0.807043,0.867781,00:09
2,0.619529,0.861232,00:10
3,0.416118,0.882672,00:09
4,0.292207,0.888422,00:09


## 모델 개선: 가중치 감소
- 모델의 가중치를 너무 크게 되지 않도록 제한하는 방법.
- 가중치가 커지면 모델이 훈련 데이터에 과적합(overfitting)될 위험이 있음.
- wd: weight decay 
- 0.1: 각 가중치를 얼마나 줄일 지 나타내는 값

In [ ]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)